In [1]:
%matplotlib notebook

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sqlalchemy import create_engine


from config import user_name, user_pass

In [3]:
#Import Streaming database from CSV
streaming_db = pd.read_csv('MoviesOnStreamingPlatforms.csv')
streaming_db.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [4]:
#Drop column "Unamed: 0"
del streaming_db["Unnamed: 0"]

In [5]:
#Drop rows where ratings are not available
streaming_df = streaming_db.dropna(subset=['IMDb', 'Rotten Tomatoes'], how='any')
streaming_df.head(5)

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [6]:
#Split Directors column into first and second director columns
streaming_df[['First Director', 'Second_Director']] = streaming_df.Directors.str.split(",", n=1, expand=True)

C:\Users\ljean\anaconda3\envs\PythonData2\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [7]:
#Split Directors column into first and second director columns
streaming_df[['Second Director', 'Third Director']] = streaming_df.Second_Director.str.split(",", n=1, expand=True)

In [8]:
#Convert streaming fields from integers to booleans
streaming_serv = streaming_df[["Netflix", "Hulu", "Prime Video", "Disney+"]].astype(bool)
streaming_serv.head(5)

,Netflix,Hulu,Prime Video,Disney+
0,True,False,False,False
1,True,False,False,False
2,True,False,False,False
3,True,False,False,False
4,True,False,True,False


In [9]:
#merge streaming databases
merged_df = pd.merge(streaming_serv, streaming_df, on="Netflix" )
merged_df.head(5)

,Netflix,Hulu_x,Prime Video_x,Disney+_x,ID,Title,Year,Age,IMDb,Rotten Tomatoes,...,Type,Directors,Genres,Country,Language,Runtime,First Director,Second_Director,Second Director,Third Director
0,True,False,False,False,1,Inception,2010,13+,8.8,87%,...,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0,Christopher Nolan,None,None,None
1,True,False,False,False,2,The Matrix,1999,18+,8.7,87%,...,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0,Lana Wachowski,Lilly Wachowski,Lilly Wachowski,None
2,True,False,False,False,3,Avengers: Infinity War,2018,13+,8.5,84%,...,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0,Anthony Russo,Joe Russo,Joe Russo,None
3,True,False,False,False,4,Back to the Future,1985,7+,8.5,96%,...,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0,Robert Zemeckis,None,None,None
4,True,False,False,False,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,...,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0,Sergio Leone,None,None,None


In [10]:
#Delete duplicate columns or those not required
del merged_df['Directors']
del merged_df['Second_Director']
del merged_df['Third Director']
del merged_df['Hulu_y']
del merged_df['Prime Video_y']
del merged_df['Disney+_y']
del merged_df['Type']
merged_df

,Netflix,Hulu_x,Prime Video_x,Disney+_x,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Genres,Country,Language,Runtime,First Director,Second Director
0,True,False,False,False,1,Inception,2010,13+,8.8,87%,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0,Christopher Nolan,None
1,True,False,False,False,2,The Matrix,1999,18+,8.7,87%,"Action,Sci-Fi",United States,English,136.0,Lana Wachowski,Lilly Wachowski
2,True,False,False,False,3,Avengers: Infinity War,2018,13+,8.5,84%,"Action,Adventure,Sci-Fi",United States,English,149.0,Anthony Russo,Joe Russo
3,True,False,False,False,4,Back to the Future,1985,7+,8.5,96%,"Adventure,Comedy,Sci-Fi",United States,English,116.0,Robert Zemeckis,None
4,True,False,False,False,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,Western,"Italy,Spain,West Germany",Italian,161.0,Sergio Leone,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15941765,False,False,False,True,16672,George of the Jungle 2,2003,7+,3.3,17%,"Adventure,Comedy,Family","United States,Australia",English,87.0,David Grossman,None
15941766,False,False,False,True,16678,That Darn Cat,1997,7+,4.7,13%,"Comedy,Crime,Family,Thriller",United States,"English,French",116.0,Robert Stevenson,None
15941767,False,False,False,True,16688,Kazaam,1996,7+,3.0,6%,"Comedy,Family,Fantasy,Musical",United States,English,93.0,Paul Michael Glaser,None
15941768,False,False,False,True,16706,Meet the Deedles,1998,7+,4.1,7%,"Comedy,Family",United States,English,93.0,Steve Boyum,None


In [11]:
#Reorder columns for easier analysis
ordered_df = merged_df[['ID','Title','Year','Age','IMDb', 'Rotten Tomatoes', 'Netflix', 'Hulu_x', 'Prime Video_x', 'Disney+_x',\
                        'First Director', 'Second Director', 'Country', 'Language','Runtime','Genres']]
ordered_df.head()

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu_x,Prime Video_x,Disney+_x,First Director,Second Director,Country,Language,Runtime,Genres
0,1,Inception,2010,13+,8.8,87%,True,False,False,False,Christopher Nolan,None,"United States,United Kingdom","English,Japanese,French",148.0,"Action,Adventure,Sci-Fi,Thriller"
1,2,The Matrix,1999,18+,8.7,87%,True,False,False,False,Lana Wachowski,Lilly Wachowski,United States,English,136.0,"Action,Sci-Fi"
2,3,Avengers: Infinity War,2018,13+,8.5,84%,True,False,False,False,Anthony Russo,Joe Russo,United States,English,149.0,"Action,Adventure,Sci-Fi"
3,4,Back to the Future,1985,7+,8.5,96%,True,False,False,False,Robert Zemeckis,None,United States,English,116.0,"Adventure,Comedy,Sci-Fi"
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,True,False,False,False,Sergio Leone,None,"Italy,Spain,West Germany",Italian,161.0,Western


In [12]:
#Clean data to reflect primary and secondary filming locations and place and populate corresponding new columns
ordered_df[['Primary Location', 'Secondary_Location']] = ordered_df.Country.str.split(",", n=1, expand=True)
ordered_df.head(5)

C:\Users\ljean\anaconda3\envs\PythonData2\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu_x,Prime Video_x,Disney+_x,First Director,Second Director,Country,Language,Runtime,Genres,Primary Location,Secondary_Location
0,1,Inception,2010,13+,8.8,87%,True,False,False,False,Christopher Nolan,None,"United States,United Kingdom","English,Japanese,French",148.0,"Action,Adventure,Sci-Fi,Thriller",United States,United Kingdom
1,2,The Matrix,1999,18+,8.7,87%,True,False,False,False,Lana Wachowski,Lilly Wachowski,United States,English,136.0,"Action,Sci-Fi",United States,None
2,3,Avengers: Infinity War,2018,13+,8.5,84%,True,False,False,False,Anthony Russo,Joe Russo,United States,English,149.0,"Action,Adventure,Sci-Fi",United States,None
3,4,Back to the Future,1985,7+,8.5,96%,True,False,False,False,Robert Zemeckis,None,United States,English,116.0,"Adventure,Comedy,Sci-Fi",United States,None
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,True,False,False,False,Sergio Leone,None,"Italy,Spain,West Germany",Italian,161.0,Western,Italy,"Spain,West Germany"


In [13]:
#Perform a split between second and third filming locations  
ordered_df[['Secondary Location', 'Third Location']] = ordered_df.Secondary_Location.str.split(",", n=1, expand=True)
ordered_df.head(5)

C:\Users\ljean\anaconda3\envs\PythonData2\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu_x,Prime Video_x,Disney+_x,First Director,Second Director,Country,Language,Runtime,Genres,Primary Location,Secondary_Location,Secondary Location,Third Location
0,1,Inception,2010,13+,8.8,87%,True,False,False,False,Christopher Nolan,None,"United States,United Kingdom","English,Japanese,French",148.0,"Action,Adventure,Sci-Fi,Thriller",United States,United Kingdom,United Kingdom,None
1,2,The Matrix,1999,18+,8.7,87%,True,False,False,False,Lana Wachowski,Lilly Wachowski,United States,English,136.0,"Action,Sci-Fi",United States,None,None,None
2,3,Avengers: Infinity War,2018,13+,8.5,84%,True,False,False,False,Anthony Russo,Joe Russo,United States,English,149.0,"Action,Adventure,Sci-Fi",United States,None,None,None
3,4,Back to the Future,1985,7+,8.5,96%,True,False,False,False,Robert Zemeckis,None,United States,English,116.0,"Adventure,Comedy,Sci-Fi",United States,None,None,None
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,True,False,False,False,Sergio Leone,None,"Italy,Spain,West Germany",Italian,161.0,Western,Italy,"Spain,West Germany",Spain,West Germany


In [14]:
#Drop Secondary_Location & Third Location
del ordered_df['Secondary_Location']
del ordered_df['Third Location']
del ordered_df['Country']
ordered_df.head(5)

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu_x,Prime Video_x,Disney+_x,First Director,Second Director,Language,Runtime,Genres,Primary Location,Secondary Location
0,1,Inception,2010,13+,8.8,87%,True,False,False,False,Christopher Nolan,None,"English,Japanese,French",148.0,"Action,Adventure,Sci-Fi,Thriller",United States,United Kingdom
1,2,The Matrix,1999,18+,8.7,87%,True,False,False,False,Lana Wachowski,Lilly Wachowski,English,136.0,"Action,Sci-Fi",United States,None
2,3,Avengers: Infinity War,2018,13+,8.5,84%,True,False,False,False,Anthony Russo,Joe Russo,English,149.0,"Action,Adventure,Sci-Fi",United States,None
3,4,Back to the Future,1985,7+,8.5,96%,True,False,False,False,Robert Zemeckis,None,English,116.0,"Adventure,Comedy,Sci-Fi",United States,None
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,True,False,False,False,Sergio Leone,None,Italian,161.0,Western,Italy,Spain


In [15]:
#Split Language column into primary and secondary languages and populate corresponding new columns
ordered_df[['Primary Language', 'Secondary_Language']] = ordered_df.Language.str.split(",", n=1, expand=True)
ordered_df.head(5)

C:\Users\ljean\anaconda3\envs\PythonData2\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu_x,Prime Video_x,Disney+_x,First Director,Second Director,Language,Runtime,Genres,Primary Location,Secondary Location,Primary Language,Secondary_Language
0,1,Inception,2010,13+,8.8,87%,True,False,False,False,Christopher Nolan,None,"English,Japanese,French",148.0,"Action,Adventure,Sci-Fi,Thriller",United States,United Kingdom,English,"Japanese,French"
1,2,The Matrix,1999,18+,8.7,87%,True,False,False,False,Lana Wachowski,Lilly Wachowski,English,136.0,"Action,Sci-Fi",United States,None,English,None
2,3,Avengers: Infinity War,2018,13+,8.5,84%,True,False,False,False,Anthony Russo,Joe Russo,English,149.0,"Action,Adventure,Sci-Fi",United States,None,English,None
3,4,Back to the Future,1985,7+,8.5,96%,True,False,False,False,Robert Zemeckis,None,English,116.0,"Adventure,Comedy,Sci-Fi",United States,None,English,None
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,True,False,False,False,Sergio Leone,None,Italian,161.0,Western,Italy,Spain,Italian,None


In [16]:
#Split Language column into primary and secondary languages and populate corresponding new columns
ordered_df[['Secondary Language', 'Third Language']] = ordered_df.Secondary_Language.str.split(",", n=1, expand=True)
ordered_df.head(5)

C:\Users\ljean\anaconda3\envs\PythonData2\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu_x,Prime Video_x,Disney+_x,...,Second Director,Language,Runtime,Genres,Primary Location,Secondary Location,Primary Language,Secondary_Language,Secondary Language,Third Language
0,1,Inception,2010,13+,8.8,87%,True,False,False,False,...,None,"English,Japanese,French",148.0,"Action,Adventure,Sci-Fi,Thriller",United States,United Kingdom,English,"Japanese,French",Japanese,French
1,2,The Matrix,1999,18+,8.7,87%,True,False,False,False,...,Lilly Wachowski,English,136.0,"Action,Sci-Fi",United States,None,English,None,None,None
2,3,Avengers: Infinity War,2018,13+,8.5,84%,True,False,False,False,...,Joe Russo,English,149.0,"Action,Adventure,Sci-Fi",United States,None,English,None,None,None
3,4,Back to the Future,1985,7+,8.5,96%,True,False,False,False,...,None,English,116.0,"Adventure,Comedy,Sci-Fi",United States,None,English,None,None,None
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,True,False,False,False,...,None,Italian,161.0,Western,Italy,Spain,Italian,None,None,None


In [17]:
#Drop Language and Secondary_language
del ordered_df['Language']
del ordered_df['Secondary_Language']
del ordered_df['Third Language']
ordered_df

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu_x,Prime Video_x,Disney+_x,First Director,Second Director,Runtime,Genres,Primary Location,Secondary Location,Primary Language,Secondary Language
0,1,Inception,2010,13+,8.8,87%,True,False,False,False,Christopher Nolan,None,148.0,"Action,Adventure,Sci-Fi,Thriller",United States,United Kingdom,English,Japanese
1,2,The Matrix,1999,18+,8.7,87%,True,False,False,False,Lana Wachowski,Lilly Wachowski,136.0,"Action,Sci-Fi",United States,None,English,None
2,3,Avengers: Infinity War,2018,13+,8.5,84%,True,False,False,False,Anthony Russo,Joe Russo,149.0,"Action,Adventure,Sci-Fi",United States,None,English,None
3,4,Back to the Future,1985,7+,8.5,96%,True,False,False,False,Robert Zemeckis,None,116.0,"Adventure,Comedy,Sci-Fi",United States,None,English,None
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,True,False,False,False,Sergio Leone,None,161.0,Western,Italy,Spain,Italian,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15941765,16672,George of the Jungle 2,2003,7+,3.3,17%,False,False,False,True,David Grossman,None,87.0,"Adventure,Comedy,Family",United States,Australia,English,None
15941766,16678,That Darn Cat,1997,7+,4.7,13%,False,False,False,True,Robert Stevenson,None,116.0,"Comedy,Crime,Family,Thriller",United States,None,English,French
15941767,16688,Kazaam,1996,7+,3.0,6%,False,False,False,True,Paul Michael Glaser,None,93.0,"Comedy,Family,Fantasy,Musical",United States,None,English,None
15941768,16706,Meet the Deedles,1998,7+,4.1,7%,False,False,False,True,Steve Boyum,None,93.0,"Comedy,Family",United States,None,English,None


In [18]:
#split the genres into seperate genre columns
ordered_df[['First Genre', 'Second Genre', 'Third Genre', 'Fourth Genre', 'Fifth Genre', 'Sixth Genre', 'Seventh Genre', 'Eigth Genre']] = ordered_df.Genres.str.split(",", expand=True)
ordered_df

C:\Users\ljean\anaconda3\envs\PythonData2\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu_x,Prime Video_x,Disney+_x,...,Primary Language,Secondary Language,First Genre,Second Genre,Third Genre,Fourth Genre,Fifth Genre,Sixth Genre,Seventh Genre,Eigth Genre
0,1,Inception,2010,13+,8.8,87%,True,False,False,False,...,English,Japanese,Action,Adventure,Sci-Fi,Thriller,None,None,None,None
1,2,The Matrix,1999,18+,8.7,87%,True,False,False,False,...,English,None,Action,Sci-Fi,None,None,None,None,None,None
2,3,Avengers: Infinity War,2018,13+,8.5,84%,True,False,False,False,...,English,None,Action,Adventure,Sci-Fi,None,None,None,None,None
3,4,Back to the Future,1985,7+,8.5,96%,True,False,False,False,...,English,None,Adventure,Comedy,Sci-Fi,None,None,None,None,None
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,True,False,False,False,...,Italian,None,Western,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15941765,16672,George of the Jungle 2,2003,7+,3.3,17%,False,False,False,True,...,English,None,Adventure,Comedy,Family,None,None,None,None,None
15941766,16678,That Darn Cat,1997,7+,4.7,13%,False,False,False,True,...,English,French,Comedy,Crime,Family,Thriller,None,None,None,None
15941767,16688,Kazaam,1996,7+,3.0,6%,False,False,False,True,...,English,None,Comedy,Family,Fantasy,Musical,None,None,None,None
15941768,16706,Meet the Deedles,1998,7+,4.1,7%,False,False,False,True,...,English,None,Comedy,Family,None,None,None,None,None,None


In [19]:
#Rename streaming fields to remove _x
streams_df = ordered_df.rename(columns={"Hulu_x":"Hulu", "Prime Video_x":"Prime Video","Disney+_x":"Disney+"})
streams_df.head(5)


,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,...,Primary Language,Secondary Language,First Genre,Second Genre,Third Genre,Fourth Genre,Fifth Genre,Sixth Genre,Seventh Genre,Eigth Genre
0,1,Inception,2010,13+,8.8,87%,True,False,False,False,...,English,Japanese,Action,Adventure,Sci-Fi,Thriller,None,None,None,None
1,2,The Matrix,1999,18+,8.7,87%,True,False,False,False,...,English,None,Action,Sci-Fi,None,None,None,None,None,None
2,3,Avengers: Infinity War,2018,13+,8.5,84%,True,False,False,False,...,English,None,Action,Adventure,Sci-Fi,None,None,None,None,None
3,4,Back to the Future,1985,7+,8.5,96%,True,False,False,False,...,English,None,Adventure,Comedy,Sci-Fi,None,None,None,None,None
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,True,False,False,False,...,Italian,None,Western,None,None,None,None,None,None,None


In [20]:
#Delete Genres column and confirm order of remaining columns
del streams_df['Genres']
streams_df.columns

Index(['ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes', 'Netflix',
       'Hulu', 'Prime Video', 'Disney+', 'First Director', 'Second Director',
       'Runtime', 'Primary Location', 'Secondary Location', 'Primary Language',
       'Secondary Language', 'First Genre', 'Second Genre', 'Third Genre',
       'Fourth Genre', 'Fifth Genre', 'Sixth Genre', 'Seventh Genre',
       'Eigth Genre'],
      dtype='object')

Convert back to Database

In [23]:
#output to CSV
streams_df.to_csv("streams_df.csv", index=False, header=True)

In [21]:
#Connect to Local Database
rds_connection_string = (f'{user_name}:{user_pass}@)localhost:5432/streaming_db')
engine = create_engine(f'postgresql://{rds_connection_string}')

In [22]:
#Use pandas to load csv converted DataFrame into database
streams_df.to_sql(name='streaming_db', con=engine, if_exists='append', index=False)

OperationalError: (psycopg2.OperationalError) could not translate host name ")localhost" to address: Unknown host

(Background on this error at: http://sqlalche.me/e/13/e3q8)